In [1]:
from langchain_openai import ChatOpenAI # type: ignore
chat = ChatOpenAI(model_name="o4-mini")
question="How many planets are there?"
# chat_response = chat.predict(question) #this method(predict) will soon be deprecated
chat_response = chat.invoke(question)
print(chat_response)


/Users/boyoon/Documents/GitHub/fullstack-gpt-2025/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/n3/6fbssxjx1x7bxcnvp_nn7g3r0000gn/T/ipykernel_25744/292855851.py:4: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat_response = chat.predict(question)


If you mean “planets in our Solar System,” there are eight, in order from the Sun:  
1. Mercury  
2. Venus  
3. Earth  
4. Mars  
5. Jupiter  
6. Saturn  
7. Uranus  
8. Neptune  

(Pluto and a handful of other bodies—Ceres, Eris, Haumea and Makemake—are classified as “dwarf planets” by the International Astronomical Union.)

If you widen the scope to the rest of the galaxy (and beyond), astronomers have now confirmed over 5 000 exoplanets orbiting other stars—and ongoing surveys suggest there may be literally billions more. The total number of planets in the universe is unknown (and almost certainly vast).


### 3.1 Predict Messages

In [7]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import HumanMessage, AIMessage, SystemMessage # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italin."),
    AIMessage(content="Ciao! Mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and France? And what's your name?")
]

# res = chat.predict_messages(messages) #this method(predict_messages) will be soon deprecated.
res = chat.invoke(messages)
print(res)

content='La distanza in linea d’aria tra Città del Messico e Parigi (capitali di Messico e Francia) è di circa 9.100 chilometri.  \nMi chiamo Paolo.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 445, 'prompt_tokens': 52, 'total_tokens': 497, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None, 'id': 'chatcmpl-BOpSZ5ykJifXMVpgHmW83n3XXMp1S', 'finish_reason': 'stop', 'logprobs': None} id='run-0879f96f-2566-4069-a11b-f08a2aff606d-0' usage_metadata={'input_tokens': 52, 'output_tokens': 445, 'total_tokens': 497, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 384}}


### 3.2 Prompt Templates

In [10]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import HumanMessage, AIMessage, SystemMessage # type: ignore
from langchain.prompts import PromptTemplate, ChatPromptTemplate # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

#PromptTempalte
template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")
prompt = template.format(country_a="Korea", country_b="France")
chat.invoke(prompt)

#ChatPromptTemplate
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao! Mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}? And what's your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.invoke(prompt)

AIMessage(content='Η ευθύγραμμη (αεροπορική) απόσταση μεταξύ Πόλης του Μεξικού και Μπανγκόκ στην Ταϊλάνδη είναι περίπου 15\xa0900 χιλιόμετρα.  \nΤο όνομά μου είναι Βοηθός.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1040, 'prompt_tokens': 52, 'total_tokens': 1092, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 960, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None, 'id': 'chatcmpl-BOpeZkfHyYLZMyPpY2cKQTLEGLPBO', 'finish_reason': 'stop', 'logprobs': None}, id='run-8f1c67e9-7445-4c06-8b18-226395c20f59-0', usage_metadata={'input_tokens': 52, 'output_tokens': 1040, 'total_tokens': 1092, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 960}})

### 3.3 OutputParser and LCEL
* LCEL : LangChain Expression Language

In [22]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import BaseOutputParser #type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma-seperated list of {max_items} in lowercase. DO NOT reply with anything else."),
    ("human", "{question}")
])


prompt = template.format_messages(
    max_items = 10,
    question="Give me 10 most popular colors."
)

colors = chat.invoke(prompt)
p.parse(colors.content)

['blue',
 'red',
 'green',
 'black',
 'white',
 'yellow',
 'purple',
 'orange',
 'pink',
 'brown']

In [23]:
# chain을 이용해서 위 코드를 개선할 수 있음
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import BaseOutputParser #type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma-seperated list of {max_items} in lowercase. DO NOT reply with anything else."),
    ("human", "{question}")
])

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5, 
    "question":"Give me the most popular pokemons."
})

['pikachu', 'charizard', 'eevee', 'mewtwo', 'bulbasaur']

### 3.4 Chaining Chains
* callbacks & streaming=True : 답변 생성이 전부 완료될 때까지 기다리지 않음. 생성되는 즉시 결과를 확인.

In [25]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore

chat = ChatOpenAI(
    model_name="o4-mini", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

#normal recipes
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy-to-follow recipes for any type of cuising with easy-to-find ingredients."),
    ("human", "I want to cook {country} food.")
])
chef_chain = chef_prompt | chat

#vegetarian recipes
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized in making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative, just say you don't know how to replace it."),
    ("human", "{recipe}")
])
veg_chef_chain = veg_chef_prompt | chat

#mixing two chains
final_chain = {"recipe":chef_chain} | veg_chef_chain

#calling OpenAI API
final_chain.invoke({
    "country":"korean"
})

Here are three classic, easy-to-find Korean dishes you can try at home. Pick one (or all three!), gather the ingredients, and follow the step‑by‑step.  

1. Kimchi Fried Rice (Kimchi Bokkeumbap)  
Ingredients (2 servings)  
 • 2 cups cold cooked rice (preferably a day old)  
 • 1 cup well-fermented kimchi, chopped (reserve 2 Tbsp kimchi juice)  
 • ½ onion, diced  
 • 1 small carrot, julienned (optional)  
 • 2 scallions, sliced  
 • 1 Tbsp gochujang (Korean red‑pepper paste) or 1 tsp gochugaru + 1 Tbsp soy sauce  
 • 1 Tbsp soy sauce  
 • 1 tsp sugar  
 • 1 tsp sesame oil  
 • 1 Tbsp neutral oil (vegetable/canola)  
 • 2 eggs  
 • Toasted sesame seeds (for garnish)  

Steps  
1. Prep: Heat a large nonstick skillet over medium–high.  
2. Aromatics: Add neutral oil → sauté onion and carrot 2–3 min until soft.  
3. Kimchi: Add chopped kimchi + 2 Tbsp kimchi juice → cook 2 min.  
4. Seasoning: Stir in gochujang (or gochugaru+soy), soy sauce, sugar → mix.  
5. Rice: Break up cold rice with

AIMessage(content='Here’s how you can turn each of these into a vegetarian (or even vegan) dish, with minimal tweaks to the original recipes.  \n\n1. Kimchi Fried Rice  \n   • Kimchi: Most store‑bought kimchis use fish sauce or shrimp paste. Look for “vegan kimchi” at Korean markets or make your own by substituting the fish ingredients with a little crushed nori (seaweed) and a splash of soy sauce or tamari in your chili paste.  \n   • Eggs (optional): If you eat eggs, keep the sunny‑side‑up as written. For a vegan version, crumble firm tofu and sauté it in a little oil, a pinch of turmeric for color, plus salt and black salt (kala namak) for that eggy aroma. Stir it in at the very end or serve on top.  \n\n2. “Mushroom Bulgogi” (in place of Beef Bulgogi)  \n   Ingredients (2–3 servings)  \n    • 300\xa0g (10\xa0oz) king oyster mushrooms (or shiitakes + cremini), thinly sliced lengthwise into “scallops”  \n    • 3\xa0Tbsp soy sauce  \n    • 2\xa0Tbsp brown sugar or maple syrup  \n    •